In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip "/content/drive/MyDrive/PLN/criticas-imdb.zip"

Archive:  /content/drive/MyDrive/PLN/criticas-imdb.zip
  inflating: criticas-imdb.csv       


In [3]:
#importando o conjunto de dados
import pandas as pd

criticas  = pd.read_csv("criticas-imdb.csv")

In [4]:
#explorando os dados
criticas.head()

,texto,sentimento
0,Eu fui e vi este filme ontem à noite depois de...,positivo
1,"O diretor do ator, Bill Paxton, segue sua prom...",positivo
2,Como um jogador de recreio com algum conhecime...,positivo
3,"Eu vi esse filme em uma prévia, e é delicioso....",positivo
4,Bill Paxton levou a verdadeira história do gol...,positivo


In [7]:
#explorando os dados
criticas.tail()

,texto,sentimento
49454,"No final do filme, senti que era muito técnico...",negativo
49455,Este é o tipo de filme que meus inimigos me as...,negativo
49456,Eu vi Descent na noite passada no Stockholm Fi...,negativo
49457,Alguns filmes que você escolhe por um quilo sã...,negativo
49458,Este é um dos filmes mais idiotas que já vi. E...,negativo


In [6]:
#dividindo o conjunto de dados em 20% para teste e 80% para treino
from sklearn.model_selection import train_test_split

criticas_treino, criticas_teste = train_test_split(criticas,
                                                   test_size=0.2,
                                                   random_state=10)

In [8]:
# observando os dados para treinamento
criticas_treino.head()

,texto,sentimento
19425,"Tornado humano de 1976 é, em muitos aspectos, ...",positivo
22621,Efeitos surpreendentes para um filme deste tem...,positivo
42506,"Este é um filme japonês, mas há um pouco de In...",negativo
6595,"Eu não sou muito para filmes de ""policial"", ma...",positivo
41850,... Ruim em ser intencionalmente ruim ... Esta...,negativo


In [11]:
# observando os dados para treinamento
len(criticas_treino)

39567

In [13]:
# observando os dados para teste
len(criticas_teste)

9892

In [14]:
#importando os vetores do NILLC para executar a vetorização do corpo de texto
!unzip "/content/drive/My Drive/PLN/cbow_s300.zip"

Archive:  /content/drive/My Drive/PLN/cbow_s300.zip
  inflating: cbow_s300.txt           


In [15]:
#lendo e observando os vetores
with open("cbow_s300.txt") as iteradorArquivo:
  for contadorLinha in range(10):
    print(next(iteradorArquivo))

929606 300

</s> -0.001667 -0.000158 -0.000026 0.001300 -0.000796 0.001527 0.000046 0.000584 0.000449 -0.000100 0.000353 0.001251 0.001069 0.000506 0.000574 0.000838 -0.000930 -0.001220 0.000317 0.001315 -0.001120 0.001373 -0.000040 -0.001580 0.000421 -0.000667 -0.001556 -0.000746 0.001604 0.001157 -0.000027 0.000354 0.000358 -0.000527 -0.000573 -0.001512 -0.001557 -0.001637 0.001617 -0.001511 -0.001022 -0.001426 0.001086 -0.001033 0.000593 0.000724 0.000627 -0.000450 -0.001140 0.000333 0.000524 0.001541 0.000284 0.000617 -0.000807 -0.000088 -0.000364 0.001126 -0.001230 -0.001138 -0.001280 0.001330 0.001257 0.000576 0.000764 0.000684 0.001008 -0.000215 -0.000629 -0.001228 -0.001557 -0.000311 -0.000246 0.000045 0.001136 -0.000645 -0.000549 0.001099 0.000858 -0.000886 0.000553 0.000303 0.001433 0.000732 0.001321 -0.000894 -0.000700 -0.000661 -0.001484 -0.000950 -0.001556 -0.000809 0.000348 -0.000068 0.000724 -0.000569 -0.000161 -0.001628 -0.001437 -0.000259 -0.000296 -0.001571 0.000149 0

In [16]:
#importando  os modelos da biblioteca gensim para vetorização com os vetores do NILLC
#implementando o modelo com o word2vec,e os vetores de 300 dimensões, no método CBOW
from gensim.models import KeyedVectors

modelo = KeyedVectors.load_word2vec_format("cbow_s300.txt")

In [17]:
#importando a biblioteca string e o NLTK punkt para tokenização do corpo de texto
import string
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [18]:
#importando numpy para operar com os vetores
import numpy as np


In [19]:
#função para tratamento dos dados e tokenização do corpus de texto
def tratarTokenizar(texto):
  texto=texto.lower()
  tokens_validos = []

  for token in nltk.word_tokenize(texto):
    if token not in string.punctuation:
      tokens_validos.append(token)

  return tokens_validos

In [20]:
#função para vetorização
def vetoresTextos(textos):
  x = len(textos)
  y = 300
  vetores = np.zeros((x,y))

  for itexto in range(x):
    tokens = tratarTokenizar(textos.iloc[itexto])
    vetores[itexto] = somaVetores(tokens)

  return vetores

In [21]:
#função para somar os vetores frutos da tokenização
def somaVetores(tokens):
  vetorSomado = np.zeros(300)
  for token in tokens:
    try:
      vetorSomado += modelo.get_vector(token)
    except:
      if token.isnumeric():
        token = "0"*len(token)
        vetorSomado += modelo.get_vector(token)
      else:
        vetorSomado += modelo.get_vector("unknown")

  return vetorSomado

In [22]:
#gerando vetores de treino e vetores de teste, observando o formato e dimensões dos vetores
vetores_treino = vetoresTextos(criticas_treino.texto)
vetores_teste = vetoresTextos(criticas_teste.texto)
print(vetores_treino.shape)
print(vetores_teste.shape)

(39567, 300)
(9892, 300)


In [23]:
#observando vetores de treino
print(vetores_treino)

[[ 2.73421795  0.311943   -3.1896231  ...  1.24842096  0.65088403
   1.60167697]
 [-0.80182408  1.91213906 -2.71754403 ...  2.95214997  1.79892198
  -1.72515507]
 [ 4.75925199  2.06169404 -2.56801903 ...  1.60172105 -1.86328699
   1.63948605]
 ...
 [ 2.30013693  1.71708792 -1.03630602 ...  1.56657088  2.42961
  -2.98386602]
 [ 4.34052694  4.23035909 -6.73330402 ... 11.77689588 -0.45343395
   2.31720613]
 [ 3.08371994  0.46220803 -3.14173901 ...  1.47123488  1.171558
   3.11910319]]


In [24]:
#observando vetores de teste
print(vetores_teste)

[[ 11.50125596   0.38926304 -16.14189713 ...  12.3522879    0.71080703
   -1.07189195]
 [  3.543043    -0.338089    -3.38593103 ...   4.40340801   0.35946697
    1.28489399]
 [ 12.03879392   6.542755   -11.0890261  ...  13.00769784   2.86570517
    1.58576021]
 ...
 [  4.12080099  -0.40596702  -0.93938813 ...   1.77653997   1.21599804
   -0.20062188]
 [  2.22691678   0.45447803 -15.69204017 ...  -1.94338914   7.60645417
   -4.27185499]
 [  4.26931801   3.35502904  -4.29404205 ...   5.02443592   1.12545601
   -1.86832695]]


In [25]:
#classificador de Regressão Logística
from sklearn.linear_model import LogisticRegression

classificador = LogisticRegression()

In [34]:
#implemnentando a classificação
classificador.fit(vetores_treino, criticas_treino.sentimento)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [35]:
#apresentando o score de classificação de 80.50%
classificador.score(vetores_teste, criticas_teste.sentimento)

0.8049939344925192

In [36]:
# as categorias previstas só poderiam ser 2 conforme o dataset, positivo e negativo
categorias_previstas = classificador.predict(vetores_treino)
categorias_previstas

array(['positivo', 'positivo', 'negativo', ..., 'negativo', 'negativo',
       'positivo'], dtype=object)

In [39]:
#enumerando quantas foram as categorias previstas
len(categorias_previstas)

39567

In [40]:
#elaborando o relatório de classificação, com 81% de precisão e 81% de recall
from sklearn.metrics import classification_report

categorias_previstas = classificador.predict(vetores_teste)
relatorio = classification_report(criticas_teste.sentimento, categorias_previstas)
print(relatorio)

              precision    recall  f1-score   support

    negativo       0.81      0.81      0.81      4991
    positivo       0.80      0.80      0.80      4901

    accuracy                           0.80      9892
   macro avg       0.80      0.80      0.80      9892
weighted avg       0.80      0.80      0.80      9892



In [41]:
#importando os vetores de 300 dimensões para word embedding com o método skip-gram
!unzip "/content/drive/My Drive/PLN/skip_s300.zip"

Archive:  /content/drive/My Drive/PLN/skip_s300.zip
  inflating: skip_s300.txt           


In [42]:
#implementando o modelo skip-gram 
modelo_skipgram = KeyedVectors.load_word2vec_format("skip_s300.txt")

In [43]:
# função para somar os vetores
import numpy as np

def somaVetores(tokens, modelo):
  vetorSomado = np.zeros(300)
  for token in tokens:
    try:
      vetorSomado += modelo.get_vector(token)
    except:
      if token.isnumeric():
        token = "0"*len(token)
        vetorSomado += modelo.get_vector(token)
      else:
        vetorSomado += modelo.get_vector("unknown")

  return vetorSomado

In [44]:
#função para tratar os dados e tokenizar o corpus
def vetoresTextos(textos, modelo):
  x = len(textos)
  y = 300
  vetores = np.zeros((x,y))

  for itexto in range(x):
    tokens = tratarTokenizar(textos.iloc[itexto])
    vetores[itexto] = somaVetores(tokens, modelo)

  return vetores

In [45]:
#gerando vetores de treino e vetores de teste com skip-gram
vetores_treino_skipgram = vetoresTextos(criticas_treino.texto, modelo_skipgram)
vetores_teste_skipgram = vetoresTextos(criticas_teste.texto, modelo_skipgram)

In [47]:
#observando as dimensões dos vetores
print(vetores_treino_skipgram.shape)
print(vetores_teste_skipgram.shape)

(39567, 300)
(9892, 300)


In [49]:
#implementando a classificação com regressão logística e o método skip-gram
classificador_skipgram = LogisticRegression(max_iter=200)
classificador_skipgram.fit(vetores_treino_skipgram, criticas_treino.sentimento)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [50]:
categorias_previstas_skipgram = classificador_skipgram.predict(vetores_teste_skipgram)
relatorio_skipgram = classification_report(criticas_teste.sentimento, categorias_previstas_skipgram)
print(relatorio_skipgram)

              precision    recall  f1-score   support

    negativo       0.81      0.81      0.81      4991
    positivo       0.81      0.81      0.81      4901

    accuracy                           0.81      9892
   macro avg       0.81      0.81      0.81      9892
weighted avg       0.81      0.81      0.81      9892



In [51]:
print(relatorio)

              precision    recall  f1-score   support

    negativo       0.81      0.81      0.81      4991
    positivo       0.80      0.80      0.80      4901

    accuracy                           0.80      9892
   macro avg       0.80      0.80      0.80      9892
weighted avg       0.80      0.80      0.80      9892



In [53]:
criticas_treino.head()

,texto,sentimento
19425,"Tornado humano de 1976 é, em muitos aspectos, ...",positivo
22621,Efeitos surpreendentes para um filme deste tem...,positivo
42506,"Este é um filme japonês, mas há um pouco de In...",negativo
6595,"Eu não sou muito para filmes de ""policial"", ma...",positivo
41850,... Ruim em ser intencionalmente ruim ... Esta...,negativo


# Podemos observar que a classificação com o  método skip-gram apresentou uma acurácia média 1% maior que o método CBOW.

# Comparando o word2vec que chegou a acurácia média de 81% e o Bag of Words utilizado nas unidades 1 e 2 que chegou a 79% de acurácia.Podemos destacar o menor tempo de execução no word2vec e menor tempo para tratamento e tokenização dos dados. A diferença na acurácia foi menor que 2%.

#Vale aqui destacar que utilizando o vetorzador TF-IDF e bigramas, na unidade 2 chegamos a uma acurácia 88%. Acurácia maior em 7 pontos percentuais para este corpus de textos, do que obtida no word2vec. Cabe para estudos futuros implementar um melhor tratamento e vetorização do , a fim de obter uma acurácia profissioanl, que possa ser utilizada para treinamento de agentes como chatbots.

#REFERÊNCIAS:
https://medium.com/@everton.tomalok/word2vec-e-sua-importância-na-etapa-de-pré-processamento-d0813acfc8ab
https://medium.com/@zafaralibagh6/a-simple-word2vec-tutorial-61e64e38a6a1

https://towardsdatascience.com/understanding-word2vec-embedding-in-practice-3e9b8985953